# Procesamiento y limpieza de los datos

In [15]:
import datetime as dt
import pandas as pd
import pathlib
import os
import time
# os.chdir(pathlib.Path.cwd())

#Conexión de google drive con el repo
from google.colab import drive
drive.mount('/content/drive')

###==========================
ruta = os.path.join("/content/drive/MyDrive","MonografiaUdeA")
os.chdir(ruta)
abspath = pathlib.Path.cwd()
print(f"Directorio de trabajo: {abspath}")
###==========================

if not os.path.exists(os.path.join(abspath, "datasets", "procesado")):
    os.makedirs(os.path.join(abspath, "datasets", "procesado"))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Directorio de trabajo: /content/drive/MyDrive/MonografiaUdeA


## Aportes hídricos

In [16]:
aportes = pd.read_csv(os.path.join(abspath,"datasets","AporEner.csv"))
aportes = aportes[['Date','Value']]
aportes = aportes.rename(columns = {'Date': 'fecha',
                                    'Value': 'aportes_energia'}).sort_values(by = 'fecha', ascending = True)

aportes['fecha'] = pd.to_datetime(aportes['fecha'])
aportes = aportes.reset_index(drop=True)
aportes.to_csv(os.path.join(abspath,"datasets", "procesado","aportes.csv"), index=False)
aportes.head(3)

,fecha,aportes_energia
0,2015-01-01,66256200.0
1,2015-01-02,64876900.0
2,2015-01-03,64367700.0


## Compras en bolsa

In [ ]:
hours = [f"Values_Hour{str(i).zfill(2)}" for i in range(1,25)]
compras_bolsa = pd.read_csv(os.path.join(abspath,"datasets","CompBolsNaciEner.csv"))

compras_bolsa['compras_bolsa'] = compras_bolsa[hours].sum(axis=1)
compras_bolsa = compras_bolsa[ ['Date'] + ['compras_bolsa']]

compras_bolsa = compras_bolsa.rename(columns = {'Date': 'fecha'}).sort_values(by = 'fecha', ascending = True)
compras_bolsa['fecha'] = pd.to_datetime(compras_bolsa['fecha'])
# compras_bolsa.to_csv('compras_bolsa_final.csv', index=False)

In [ ]:


#%%
compras_contrato = pd.read_csv('compras_contrato.csv')
compras_contrato['compras_contrato'] = compras_contrato[hours].sum(axis=1)
compras_contrato = compras_contrato[ ['Date'] + ['compras_contrato']]

compras_contrato = compras_contrato.rename(columns = {'Date': 'fecha'}).sort_values(by = 'fecha', ascending = True)
compras_contrato['fecha'] = pd.to_datetime(compras_contrato['fecha'])
# compras_contrato.to_csv('compras_contrato_final.csv', index=False)

#%%
demanda = pd.read_csv('demanda_sin.csv')
# demanda['demanda'] = demanda[hours].sum(axis=1)
# demanda = demanda[ ['Date'] + ['demanda']]
demanda = demanda[['Date','Value']]


demanda = demanda.rename(columns = {'Date': 'fecha',
                                    'Value': 'demanda'}).sort_values(by = 'fecha', ascending = True)
demanda['fecha'] = pd.to_datetime(demanda['fecha'])
# demanda.to_csv('demanda_final.csv', index=False)

#%%
disponibilidad_declarada = pd.read_csv('disponibilidad_declarada.csv')
disponibilidad_declarada['disponibilidad_declarada'] = disponibilidad_declarada[hours].mean(axis=1)
disponibilidad_declarada = disponibilidad_declarada.groupby('Date')['disponibilidad_declarada'].mean().reset_index()
disponibilidad_declarada = disponibilidad_declarada[ ['Date'] + ['disponibilidad_declarada']]
disponibilidad_declarada = disponibilidad_declarada.rename(columns = {'Date': 'fecha'}).sort_values(by = 'fecha', ascending = True)
disponibilidad_declarada['fecha'] = pd.to_datetime(disponibilidad_declarada['fecha'])

#%%

df_final = pd.merge(aportes, compras_bolsa, how = 'left', on = 'fecha')
df_final = pd.merge(df_final, compras_contrato, how = 'left', on = 'fecha')
df_final = pd.merge(df_final, demanda, how = 'left', on = 'fecha')


#%%
